In [ ]:
Predict Employee salreies using decision tree regression
 - Year of expericence
 - Education level
 - Number of certifications 
    
1️⃣ Create an in-memory dataset (employee salary data)
2️⃣ Perform feature engineering (convert features into a vector)
3️⃣ Train a Decision Tree Regression Model
4️⃣ Make Predictions
5️⃣ Evaluate Model Performance
    
    

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import DecisionTreeRegressor

spark = SparkSession.builder.appName("EmployeeSalaryPrediction").getOrCreate()

data = [
    (1, 2, 1, 40000),   # 1 year exp, Bachelor's, 1 certification -> $40,000
    (3, 2, 2, 50000),   # 3 years exp, Bachelor's, 2 certs -> $50,000
    (5, 3, 3, 65000),   # 5 years exp, Master's, 3 certs -> $65,000
    (7, 3, 4, 80000),   # 7 years exp, Master's, 4 certs -> $80,000
    (10, 4, 5, 100000), # 10 years exp, PhD, 5 certs -> $100,000
    (2, 1, 0, 35000),   # 2 years exp, High School, 0 certs -> $35,000
]


columns = ["Experience", "Education", "Certifications", "Salary"]
df = spark.createDataFrame(data, columns)

assembler = VectorAssembler(inputCols=["Experience", "Education", "Certifications"], outputCol="features")
df = assembler.transform(df).select("features", col("Salary").alias("label"))

train_df, test_df = df.randomSplit([0.8, 0.2], seed=42)


dt = DecisionTreeRegressor(featuresCol="features", labelCol="label")
model = dt.fit(train_df)

predictions = model.transform(test_df)
predictions.select("features", "label", "prediction").show()

spark.stop()

25/02/13 11:13:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/13 11:13:54 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/02/13 11:13:55 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.
25/02/13 11:14:19 WARN DecisionTreeMetadata: DecisionTree reducing maxBins from 32 to 4 (= number of training instances)


+-------------+-----+----------+
|     features|label|prediction|
+-------------+-----+----------+
|[5.0,3.0,3.0]|65000|   50000.0|
|[2.0,1.0,0.0]|35000|   40000.0|
+-------------+-----+----------+

